In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from dataloader import get_dataloaders
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from transformer import Transformer,TransformerEncoder,TransformerDecoder
import utils
import pickle
nltk.download('punkt')  
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

[nltk_data] Downloading package punkt to
[nltk_data]     /home/wicaksonolxn/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
BATCH_SIZE = 8 # butuh lebih banyak update 
DATA_PATH = "dataset/"  
train_loader, val_loader, test_loader = get_dataloaders(
    data_path=DATA_PATH, 
    source_lang="min", 
    target_lang="eng", 
    batch_size=BATCH_SIZE, 
    device=device
)
SRC_VOCAB_SIZE = 5000  
TGT_VOCAB_SIZE = 5000  
DROPOUT = 0.15      
N_HEADS = 2
N_LAYERS = 4       
FFN_HIDDEN = 256      
D_MODEL = 128        

EPOCHS = 250
SAVE_DIR = "saved"
encoder = TransformerEncoder(SRC_VOCAB_SIZE,D_MODEL,N_LAYERS,N_HEADS,FFN_HIDDEN,DROPOUT,device)
decoder = TransformerDecoder(TGT_VOCAB_SIZE,D_MODEL,N_LAYERS,N_HEADS,FFN_HIDDEN,DROPOUT,device)
model = Transformer(encoder,decoder,device,utils.PAD_TOKEN).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-5)
criterion = nn.CrossEntropyLoss(ignore_index=utils.PAD_TOKEN) 
print("Model initialized on:", device)


Model initialized on: cuda


In [3]:
tokens = {
    "Padding": utils.PAD_TOKEN,
    "Start of Sequence": utils.SOS_TOKEN,
    "End of Sequence": utils.EOS_TOKEN,
    "Unknown": utils.UNK_TOKEN
}
for i, batch in enumerate(train_loader):
    if i < 1:
        src = batch["src"]
        tgt = batch["tgt"]
        ss,fss=src[0,:],src.shape
        st,fst=tgt[0,:],tgt.shape
        print(fss,ss)
    for name, token in tokens.items():
        print(f"{name}: {token}")
    else:
        break


torch.Size([8, 52]) tensor([  1,  55, 134, 987,  58,  47, 988, 989, 239, 126,  18,   7, 990, 991,
         23, 992,  40,  93, 993,   8, 521,  27, 355, 987, 239, 994,  27,   2,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0])
Padding: 0
Start of Sequence: 1
End of Sequence: 2
Unknown: 3


/home/wicaksonolxn/Documents/learnNLP/3.seq2seqTransformer/src/dataloader/_collate_fn.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  src_batch = [torch.tensor(item['src'], dtype=torch.long) for item in batch]
/home/wicaksonolxn/Documents/learnNLP/3.seq2seqTransformer/src/dataloader/_collate_fn.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tgt_batch = [torch.tensor(item['tgt'], dtype=torch.long) for item in batch]


testing input , is it correct

In [4]:
for i,batch in enumerate(train_loader):
    if i <1:
        src_batch = batch['src'].to(device)
        tgt_batch = batch['tgt'].to(device)
        output, _ = model(src_batch, tgt_batch[:, :-1]) 
        output_dim = output.shape[-1]
        output = output.reshape(-1, output_dim)
        tgt_y = tgt_batch[:,1:].contiguous().view(-1)
        print(tgt_y)

tensor([ 431,    6,  456,   17,  542,   28, 1188,   22,  156, 1164,  174, 1189,
          39, 1190,  416,  115,  840,  742,   61, 1191,   30,  282,   22,  471,
          28,    2,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,  376,  322,  161,  107,  226, 1382,   28,   10,  135,
          70, 1383,   92, 1384,   39,   10,  135,   70, 1295,   36, 1385,    2,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,  241,  769,   31,   21,  623,   10,
          73,  388, 1990,  524,  549, 1255, 1991,   73, 1992, 1993,  163,   22,
         471,  132,   73,   94,  760,  760,  847,   28,   22, 1994, 1995,   39,
          22, 1296,    6,   22, 1996, 1269,  115,  283, 1997,   76,   87, 1998,
         858,  127,  132,   28,    2,   


        output,_= model(src_batch, tgt_batch)
        output_dim = output.shape[-1]
        output = output[:, :-1, :].reshape(-1, output_dim)
        tgt_y  = tgt_batch[:, 1:].reshape(-1)
        pth = "dataset"
src = "min"
tgt = "eng"
tp  = os.path.join(pth,f"{src}_{tgt}")
input_dic_path = os.path.join(tp, "input_dic.pkl")
output_dic_path = os.path.join(tp, "output_dic.pkl")
with open(input_dic_path, "rb") as f:
    inp = pickle.load(f)
with open(output_dic_path, "rb") as f:
    output_dictionary = pickle.load(f)

In [ ]:
import os
import torch
from tqdm import tqdm

os.makedirs(SAVE_DIR, exist_ok=True)
best_val_loss = float("inf")
best_model_path = None
for epoch in range(1, EPOCHS + 1):
    print(f"Epoch {epoch}/{EPOCHS}")
    model.train()
    total_train_loss = 0.0
    train_bar = tqdm(train_loader, desc="🚀 Training", 
                leave=False, total=len(train_loader))
    for batch in train_loader:
        optimizer.zero_grad()
        src_batch = batch['src'].to(device)
        tgt_batch = batch['tgt'].to(device)
        
        output, _ = model(src_batch, tgt_batch[:, :-1]) 
        output_dim = output.shape[-1]
        output = output.reshape(-1, output_dim)
        tgt_y = tgt_batch[:,1:].contiguous().view(-1)

        loss = criterion(output, tgt_y)
        loss.backward()
        optimizer.step()
        total_train_loss += loss.item()
        train_bar.set_postfix(loss=f"{loss.item():.4f}")

    avg_train_loss = total_train_loss / len(train_loader)
    model.eval()
    total_val_loss = 0.0
    val_bar = tqdm(val_loader, desc="🚀 Validation",
              leave=True, total=len(val_loader))
    with torch.no_grad():
        for batch in val_bar:
            src_batch = batch['src'].to(device)
            tgt_batch = batch['tgt'].to(device)
            
            output, _ = model(src_batch, tgt_batch[:, :-1]) 
            output_dim = output.shape[-1]
            output = output.reshape(-1, output_dim)
            tgt_y = tgt_batch[:,1:].contiguous().view(-1)

            loss = criterion(output, tgt_y)
            total_val_loss += loss.item()
            val_bar.set_postfix(loss=f"{loss.item():.4f}")
    
    avg_val_loss = total_val_loss / len(val_loader)
    print(f"[Epoch {epoch}] Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")
    if avg_val_loss < best_val_loss:
        if best_model_path and os.path.exists(best_model_path):
            os.remove(best_model_path)
        best_val_loss = avg_val_loss
        best_model_path = os.path.join(SAVE_DIR, "best.pt")
        torch.save(model.state_dict(), best_model_path)
        print(f"  -> New best model saved at {best_model_path}")


Epoch 1/250


🚀 Validation: 100%|██████████| 21/21 [00:00<00:00, 142.08it/s, loss=8.4913]


[Epoch 1] Train Loss: 8.5771 | Val Loss: 8.5268
  -> New best model saved at saved/best.pt
Epoch 2/250


🚀 Validation: 100%|██████████| 21/21 [00:00<00:00, 169.74it/s, loss=8.1761]


[Epoch 2] Train Loss: 8.2827 | Val Loss: 8.2291
  -> New best model saved at saved/best.pt
Epoch 3/250


...

🚀 Validation: 100%|██████████| 21/21 [00:00<00:00, 82.82it/s, loss=6.5408]


[Epoch 75] Train Loss: 5.5652 | Val Loss: 6.5606
Epoch 76/250


🚀 Validation: 100%|██████████| 21/21 [00:00<00:00, 117.43it/s, loss=6.5351]


[Epoch 76] Train Loss: 5.5509 | Val Loss: 6.5558
  -> New best model saved at saved/best.pt
Epoch 77/250


🚀 Training:   0%|          | 0/102 [00:00<?, ?it/s, loss=5.2665]

## Bleu Score

loading the best model !


In [ ]:
import os
import torch
from tqdm import tqdm
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction
SRC_VOCAB_SIZE = 5000  
TGT_VOCAB_SIZE = 5000  
DROPOUT = 0.15      
N_HEADS = 2
N_LAYERS = 4       
FFN_HIDDEN = 256      
D_MODEL = 128    
encoder = TransformerEncoder(SRC_VOCAB_SIZE, D_MODEL, N_LAYERS, N_HEADS, FFN_HIDDEN, DROPOUT, device)
decoder = TransformerDecoder(TGT_VOCAB_SIZE, D_MODEL, N_LAYERS, N_HEADS, FFN_HIDDEN, DROPOUT, device)
best_model = Transformer(encoder, decoder, device, utils.PAD_TOKEN).to(device)
best_model.load_state_dict(torch.load(os.path.join(SAVE_DIR, "best.pt")))
print("Loaded best model for testing!")
pth = "dataset"
src = "min"
tgt = "eng"
tp  = os.path.join(pth, f"{src}_{tgt}")
with open(os.path.join(tp, "input_dic.pkl"),  "rb") as f:
    input_lang_dic = pickle.load(f)
with open(os.path.join(tp, "output_dic.pkl"), "rb") as f:
    output_lang_dic = pickle.load(f)
# ---------------------------------------------------------------------
# 4) Evaluate function
# ---------------------------------------------------------------------
def evaluate(model, valid_dataloader):
    model.eval()
    
    total_loss = 0.0
    all_bleu   = []
    with torch.no_grad():
        for batch_num, batch in enumerate(valid_dataloader):
            src_batch = batch["src"].to(device)
            tgt_batch = batch["tgt"].to(device)
            output, _ = model(src_batch, tgt_batch[:, :-1])
            out_dim = output.size(-1)  
            output_2d = output.contiguous().view(-1, out_dim)
            tgt_2d = tgt_batch[:, 1:].contiguous().view(-1)
            
            loss = criterion(output_2d, tgt_2d)
            total_loss += loss.item()
            batch_size = src_batch.size(0)
            batch_bleu = []
            for i in range(batch_size):
                ref_tokens = utils.detokenize(tgt_batch[i].tolist(), output_lang_dic)
                pred_ids   = output[i].argmax(dim=1)  # shape [seq_len-1]
                hyp_tokens = utils.detokenize(pred_ids.tolist(), output_lang_dic)
                bleu_score = utils.get_bleu(hyp_tokens.split(), ref_tokens.split())
                batch_bleu.append(bleu_score)
            all_bleu.append(sum(batch_bleu) / len(batch_bleu))
    epoch_loss = total_loss / len(valid_dataloader)
    epoch_bleu = sum(all_bleu) / len(all_bleu)
    return epoch_loss, epoch_bleu
valid_loss, valid_bleu = evaluate(best_model, val_loader)
print(f"Validation Loss = {valid_loss:.4f} | BLEU = {valid_bleu:.2f}")

Loaded best model for testing!
Validation Loss = 8.7341 | BLEU = 89.00


## Translasi

In [ ]:
import os
import pickle
import torch
from utils import tokenize,detokenize
import torch
import utils  # For PAD_TOKEN, SOS_TOKEN, EOS_TOKEN, UNK_TOKEN, etc.
def translate_sentence(token_ids,input_dic, output_dic, model, device, max_len=50):
    model.eval()
    
    input_tensor = torch.LongTensor(token_ids).unsqueeze(0).to(device)
    src_mask = model.make_input_mask(input_tensor)
    
    print("Source tokens:", token_ids)
    print("Decoded tokens:", [input_dic.index2word[x] for x in token_ids])
    with torch.no_grad():
        encoded_input = model.encoder(input_tensor, src_mask)
    predicted_tokens = [utils.SOS_TOKEN]
    for _ in range(max_len):
        tgt_tensor = torch.LongTensor(predicted_tokens).unsqueeze(0).to(device)
        tgt_mask = model.make_target_mask(tgt_tensor)
        with torch.no_grad():
            output, attention = model.decoder(tgt_tensor, encoded_input, tgt_mask, src_mask)
        next_token = output[0, -1].argmax(dim=-1).item()
        predicted_tokens.append(next_token)
        
        if next_token == utils.EOS_TOKEN:
            break
    translation = ' '.join(output_dic.index2word[idx] 
                           for idx in predicted_tokens[1:] 
                           if idx not in [utils.SOS_TOKEN, utils.EOS_TOKEN, utils.PAD_TOKEN])
    return translation
sentences = [
    "Ambo mancari awaknyo besok",
    "Alun salama, apo kabar?",
    "Dunsanak ka rumah gadang",
    "Urang minang manarimo tradisi",
    "Apo ado di pasar?"
]
tokenized_inputs = []
for s in sentences:
    tokens = utils.tokenize(s, input_lang_dic, utils.MAX_SENT_LEN)
    tokenized_inputs.append(tokens)
    
for idx, token_ids in enumerate(tokenized_inputs):
    translation = translate_sentence(
        token_ids,
        input_lang_dic,
        output_lang_dic,
        best_model,
        device,
        utils.MAX_SENT_LEN
    )
    print(f"Original: {sentences[idx]}")
    print(f"Translated: {translation}\n")

Source tokens: [1, 3, 573, 3, 3, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Decoded tokens: ['SOS', 'UNK', 'mancari', 'UNK', 'UNK', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD']
Original: Ambo mancari awaknyo besok
Translated: 

Source tokens: [1, 3, 3, 425, 3, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Decoded tokens: ['SOS', 'UNK', 'UNK', 'apo', 'UNK', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PA

In [ ]:
print("Index to word mapping (first 10):")
for i in range(10):
    print(i, output_lang_dic.index2word[i])


Index to word mapping (first 10):
0 PAD
1 SOS
2 EOS
3 UNK
4 enjoy
5 instalment
6 for
7 up
8 to
9 months


In [ ]:
print("Dictionary size:", len(input_lang_dic.word2index))
print("Special tokens in the dictionary:")
for idx in range(4):
    print(idx, input_lang_dic.index2word[idx])


Dictionary size: 2183
Special tokens in the dictionary:
0 PAD
1 SOS
2 EOS
3 nikmati


In [ ]:
for name, param in best_model.named_parameters():
    if "word_embedding" in name:
        print(name, param.shape)

encoder.word_embedding.weight torch.Size([5000, 128])
decoder.word_embedding.weight torch.Size([5000, 128])


In [ ]:
# import os
# import pickle
# import torch

# from utils import tokenize,detokenize
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# # -----------------------------------------------------------------------------
# # 1. Load the best model for testing
# # -----------------------------------------------------------------------------

# # Create encoder, decoder, and the full Transformer model.
# encoder = TransformerEncoder(SRC_VOCAB_SIZE, D_MODEL, N_LAYERS, N_HEADS, FFN_HIDDEN, DROPOUT, device)
# decoder = TransformerDecoder(TGT_VOCAB_SIZE, D_MODEL, N_LAYERS, N_HEADS, FFN_HIDDEN, DROPOUT, device)
# best_model = Transformer(encoder, decoder, device, utils.PAD_TOKEN).to(device)

# # Load the best model state
# model_path = os.path.join(SAVE_DIR, "best.pt")
# best_model.load_state_dict(torch.load(model_path, map_location=device))
# print("Loaded best model for testing!")
# # -----------------------------------------------------------------------------
# # 2. Load the input and output dictionaries
# # -----------------------------------------------------------------------------
# pth = "dataset"
# src = "min"
# tgt = "eng"
# tp  = os.path.join(pth,f"{src}_{tgt}")
# input_dic_path = os.path.join(tp, "input_dic.pkl")
# output_dic_path = os.path.join(tp, "output_dic.pkl")
# with open(input_dic_path, "rb") as f:
#     input_dictionary = pickle.load(f)
# with open(output_dic_path, "rb") as f:
#     output_dictionary = pickle.load(f)

# minang_sentences = [
#     "Ambo mancari awaknyo besok",
#     "Alun salama, apo kabar?",
#     "Dunsanak ka rumah gadang",
#     "Urang minang manarimo tradisi",
#     "Apo ado di pasar?"
# ]

# for sentence in minang_sentences:
#     translation = translate_sentence(sentence, best_model, input_dictionary, output_dictionary)
#     print(f"Original:   {sentence}")
#     print(f"Translated: {translation}\n")


# Translasi: 
Ambo mancari awaknyo besok

    Bahasa Indonesia: Saya akan mencarimu besok.
    English: I will look for you tomorrow.

Alun salama, apo kabar?

    Bahasa Indonesia: Halo, apa kabar?
    English: Hello, how are you?

Dunsanak ka rumah gadang

    Bahasa Indonesia: Saudara, mari ke rumah gadang.
    English: Relatives, let's go to the traditional house.

Urang minang manarimo tradisi

    Bahasa Indonesia: Orang Minang menerima tradisi.
    English: Minangkabau people embrace tradition.

Apo ado di pasar?

    Bahasa Indonesia: Apa ada di pasar?
    English: What's there in the market?